# Email Phishing Dataset - Data Preprocessing

This notebook performs data cleaning and preprocessing for the BERT-LSTM phishing detection model.

## Steps:
1. Load all datasets
2. Filter out garbage labels (keep only 0 and 1)
3. Standardize schema with metadata fields (sender, receiver, date, urls)
4. Combine all clean datasets
5. Text preprocessing and cleaning
6. Train/validation/test split
7. Save processed data

## Final Schema:
- **sender** - Email sender (null for datasets without metadata)
- **receiver** - Email receiver (null for datasets without metadata)
- **date** - Email date (null for datasets without metadata)
- **text** - Combined subject + body
- **urls** - URL information (null for datasets without metadata)
- **label** - 0 (legitimate) or 1 (phishing)
- **source** - Original dataset name

## 1. Setup

In [10]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print('Libraries imported successfully')

Libraries imported successfully


In [11]:
# Define paths
DATA_PATH = Path('../../../data/unprocessed/email-detection/Seven Emails phishing dataset')
PROCESSED_PATH = Path('../../../data/processed/email-detection')

# Create processed data directory
PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

print(f'Raw data path: {DATA_PATH}')
print(f'Processed data path: {PROCESSED_PATH}')

Raw data path: ..\..\..\data\unprocessed\email-detection\Seven Emails phishing dataset
Processed data path: ..\..\..\data\processed\email-detection


## 2. Load All Datasets

In [12]:
def load_dataset(file_path, file_name):
    """Load dataset with appropriate error handling"""
    try:
        if file_name in ['TREC-05.csv', 'TREC-06.csv']:
            df = pd.read_csv(
                file_path,
                on_bad_lines='skip',
                engine='python',
                encoding='utf-8',
                quoting=1
            )
            return df, 'loaded_with_skipped_lines'
        else:
            df = pd.read_csv(file_path)
            return df, 'loaded_successfully'
    except Exception as e:
        return None, f'error: {str(e)}'

csv_files = ['Assassin.csv', 'CEAS-08.csv', 'Enron.csv', 'Ling.csv', 'TREC-05.csv', 'TREC-06.csv', 'TREC-07.csv']
datasets = {}

print('Loading datasets...')
print('='*70)
for file in csv_files:
    df, status = load_dataset(DATA_PATH / file, file)
    if df is not None:
        name = file.replace('.csv', '')
        datasets[name] = df
        print(f'✓ {file:20s} - {len(df):>7,} rows - {status}')
    else:
        print(f'✗ {file:20s} - {status}')

print(f'\nTotal datasets loaded: {len(datasets)}')
print(f'Total raw emails: {sum(len(df) for df in datasets.values()):,}')

Loading datasets...
✓ Assassin.csv         -   5,809 rows - loaded_successfully
✓ CEAS-08.csv          -  39,154 rows - loaded_successfully
✓ Enron.csv            -  29,767 rows - loaded_successfully
✓ Ling.csv             -   2,859 rows - loaded_successfully
✓ TREC-05.csv          -  59,015 rows - loaded_with_skipped_lines
✓ TREC-06.csv          -  16,439 rows - loaded_with_skipped_lines
✓ TREC-07.csv          -  53,757 rows - loaded_successfully

Total datasets loaded: 7
Total raw emails: 206,800


## 3. Filter and Clean Labels

In [13]:
def clean_labels(df, dataset_name):
    """Filter dataset to keep only valid binary labels (0 and 1)"""
    original_count = len(df)
    
    # Convert labels to numeric, coerce errors to NaN
    df['label'] = pd.to_numeric(df['label'], errors='coerce')
    
    # Filter to keep only 0 and 1
    df_clean = df[df['label'].isin([0, 1])].copy()
    
    # Convert to integer
    df_clean['label'] = df_clean['label'].astype(int)
    
    removed = original_count - len(df_clean)
    
    print(f'{dataset_name:15s} - Original: {original_count:>6,} | Clean: {len(df_clean):>6,} | Removed: {removed:>5,} ({removed/original_count*100:>5.2f}%)')
    
    return df_clean

print('Cleaning labels (keeping only 0 and 1)...')
print('='*70)

cleaned_datasets = {}
for name, df in datasets.items():
    cleaned_datasets[name] = clean_labels(df, name)

total_clean = sum(len(df) for df in cleaned_datasets.values())
total_removed = sum(len(datasets[name]) - len(cleaned_datasets[name]) for name in datasets.keys())

print('='*70)
print(f'Total clean emails: {total_clean:,}')
print(f'Total removed: {total_removed:,}')
print(f'Retention rate: {total_clean/(total_clean+total_removed)*100:.2f}%')

Cleaning labels (keeping only 0 and 1)...
Assassin        - Original:  5,809 | Clean:  5,809 | Removed:     0 ( 0.00%)
CEAS-08         - Original: 39,154 | Clean: 39,154 | Removed:     0 ( 0.00%)
Enron           - Original: 29,767 | Clean: 29,767 | Removed:     0 ( 0.00%)
Ling            - Original:  2,859 | Clean:  2,859 | Removed:     0 ( 0.00%)
TREC-05         - Original: 59,015 | Clean: 55,210 | Removed: 3,805 ( 6.45%)
TREC-06         - Original: 16,439 | Clean: 16,382 | Removed:    57 ( 0.35%)
TREC-07         - Original: 53,757 | Clean: 53,757 | Removed:     0 ( 0.00%)
Total clean emails: 202,938
Total removed: 3,862
Retention rate: 98.13%


## 4. Standardize Schema and Combine Datasets

In [16]:
def standardize_schema(df, dataset_name):
    """Standardize schema to include metadata fields: sender, receiver, date, subject, body, urls, label, source"""
    
    # Ensure subject and body exist
    if 'subject' not in df.columns or 'body' not in df.columns:
        print(f'Warning: {dataset_name} missing subject or body columns')
        return None
    
    # Start with required columns
    df_std = pd.DataFrame()
    
    # Add metadata columns (if they exist, otherwise add as None)
    df_std['sender'] = df['sender'] if 'sender' in df.columns else None
    df_std['receiver'] = df['receiver'] if 'receiver' in df.columns else None
    df_std['date'] = df['date'] if 'date' in df.columns else None
    df_std['urls'] = df['urls'] if 'urls' in df.columns else None
    
    # Add text columns
    df_std['subject'] = df['subject']
    df_std['body'] = df['body']
    
    # Add label and source
    df_std['label'] = df['label']
    df_std['source'] = dataset_name
    
    return df_std

print('Standardizing schemas...')
print('='*70)

standardized_datasets = []
for name, df in cleaned_datasets.items():
    df_std = standardize_schema(df, name)
    if df_std is not None:
        standardized_datasets.append(df_std)
        has_metadata = 'sender' in df.columns and 'urls' in df.columns
        schema_type = 'Full (with metadata)' if has_metadata else 'Minimal (text only)'
        print(f'✓ {name:15s} - {len(df_std):>6,} emails - {schema_type}')

# Combine all datasets
print('\nCombining datasets...')
df_combined = pd.concat(standardized_datasets, ignore_index=True)

print(f'\nCombined dataset shape: {df_combined.shape}')
print(f'Columns: {list(df_combined.columns)}')

# Check metadata availability
print(f'\nMetadata availability:')
sender_count = df_combined['sender'].notna().sum()
receiver_count = df_combined['receiver'].notna().sum()
date_count = df_combined['date'].notna().sum()
urls_count = df_combined['urls'].notna().sum()
total_count = len(df_combined)

sender_pct = sender_count / total_count * 100
receiver_pct = receiver_count / total_count * 100
date_pct = date_count / total_count * 100
urls_pct = urls_count / total_count * 100

print(f'  Emails with sender: {sender_count:,} ({sender_pct:.1f}%)')
print(f'  Emails with receiver: {receiver_count:,} ({receiver_pct:.1f}%)')
print(f'  Emails with date: {date_count:,} ({date_pct:.1f}%)')
print(f'  Emails with urls: {urls_count:,} ({urls_pct:.1f}%)')

print(f'\nLabel distribution:')
print(df_combined['label'].value_counts().sort_index())
print(f'\nSource distribution:')
print(df_combined['source'].value_counts())

Standardizing schemas...
✓ Assassin        -  5,809 emails - Full (with metadata)
✓ CEAS-08         - 39,154 emails - Full (with metadata)
✓ Enron           - 29,767 emails - Minimal (text only)
✓ Ling            -  2,859 emails - Minimal (text only)
✓ TREC-05         - 55,210 emails - Full (with metadata)
✓ TREC-06         - 16,382 emails - Full (with metadata)
✓ TREC-07         - 53,757 emails - Full (with metadata)

Combining datasets...

Combined dataset shape: (202938, 8)
Columns: ['sender', 'receiver', 'date', 'urls', 'subject', 'body', 'label', 'source']

Metadata availability:
  Emails with sender: 170,030 (83.8%)
  Emails with receiver: 167,034 (82.3%)
  Emails with date: 168,491 (83.0%)
  Emails with urls: 170,312 (83.9%)

Label distribution:
label
0    108624
1     94314
Name: count, dtype: int64

Source distribution:
source
TREC-05     55210
TREC-07     53757
CEAS-08     39154
Enron       29767
TREC-06     16382
Assassin     5809
Ling         2859
Name: count, dtype: int64


## 5. Handle Missing Values

In [17]:
print('Missing values before cleaning:')
print('='*70)
print(df_combined.isnull().sum())
print(f'\nRows with missing subject: {df_combined["subject"].isnull().sum()}')
print(f'Rows with missing body: {df_combined["body"].isnull().sum()}')

# Fill missing values with empty string
df_combined['subject'] = df_combined['subject'].fillna('')
df_combined['body'] = df_combined['body'].fillna('')

# Drop rows where both subject and body are empty
before_drop = len(df_combined)
df_combined = df_combined[
    (df_combined['subject'].astype(str).str.strip() != '') | 
    (df_combined['body'].astype(str).str.strip() != '')
].copy()
after_drop = len(df_combined)

print(f'\nRows dropped (empty subject AND body): {before_drop - after_drop}')
print(f'Remaining rows: {after_drop:,}')

Missing values before cleaning:
sender      32908
receiver    35904
date        34447
urls        32626
subject      2405
body            3
label           0
source          0
dtype: int64

Rows with missing subject: 2405
Rows with missing body: 3

Rows dropped (empty subject AND body): 0
Remaining rows: 202,938


## 6. Text Preprocessing

In [18]:
def preprocess_text(text):
    """Basic text preprocessing"""
    if pd.isna(text) or text == '':
        return ''
    
    # Convert to string
    text = str(text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    # Remove very long sequences of repeated characters (likely noise)
    text = re.sub(r'(.)\1{10,}', r'\1\1\1', text)
    
    return text

print('Preprocessing text...')
print('='*70)

# Apply preprocessing
df_combined['subject_clean'] = df_combined['subject'].apply(preprocess_text)
df_combined['body_clean'] = df_combined['body'].apply(preprocess_text)

# Combine subject and body for model input
df_combined['text'] = df_combined['subject_clean'] + ' ' + df_combined['body_clean']
df_combined['text'] = df_combined['text'].str.strip()

# Calculate text statistics
df_combined['text_length'] = df_combined['text'].str.len()
df_combined['word_count'] = df_combined['text'].str.split().str.len()

print('Text preprocessing complete!')
print(f'\nText length statistics:')
print(df_combined['text_length'].describe())
print(f'\nWord count statistics:')
print(df_combined['word_count'].describe())

Preprocessing text...
Text preprocessing complete!

Text length statistics:
count    202938.000000
mean       1613.441992
std        3981.321365
min           3.000000
25%         360.000000
50%         774.000000
75%        1693.000000
max      641703.000000
Name: text_length, dtype: float64

Word count statistics:
count    202938.000000
mean        258.818925
std         611.793552
min           1.000000
25%          58.000000
50%         128.000000
75%         281.000000
max       63380.000000
Name: word_count, dtype: float64


In [19]:
# Check for very short or very long emails
print('Email length distribution:')
print('='*70)
print(f'Very short (< 10 chars): {(df_combined["text_length"] < 10).sum():,}')
print(f'Short (10-100 chars): {((df_combined["text_length"] >= 10) & (df_combined["text_length"] < 100)).sum():,}')
print(f'Medium (100-1000 chars): {((df_combined["text_length"] >= 100) & (df_combined["text_length"] < 1000)).sum():,}')
print(f'Long (1000-5000 chars): {((df_combined["text_length"] >= 1000) & (df_combined["text_length"] < 5000)).sum():,}')
print(f'Very long (> 5000 chars): {(df_combined["text_length"] >= 5000).sum():,}')

# Sample some texts
print('\nSample emails:')
print('='*70)
print('\nLegitimate email sample:')
print(df_combined[df_combined['label']==0]['text'].iloc[0][:200] + '...')
print('\nPhishing email sample:')
print(df_combined[df_combined['label']==1]['text'].iloc[0][:200] + '...')

Email length distribution:
Very short (< 10 chars): 6
Short (10-100 chars): 5,940
Medium (100-1000 chars): 114,662
Long (1000-5000 chars): 72,045
Very long (> 5000 chars): 10,285

Sample emails:

Legitimate email sample:
Re: New Sequences Window Date: Wed, 21 Aug 2002 10:54:46 -0500 From: Chris Garrigues Message-ID: <1029945287.4797.TMDA@deepeddy.vircio.com> | I can't reproduce this error. For me it is very repeatable...

Phishing email sample:
Life Insurance - Why Pay More? * * * --- * * * <= /TR> Save up to 70% on Life Insurance. Why Spend More Than You Have To? Life Quote Savings __** _**__ **Ensurin= g your family's financial security is...


## 7. Final Dataset Summary

In [20]:
print('FINAL DATASET SUMMARY')
print('='*70)
print(f'Total emails: {len(df_combined):,}')
print(f'Shape: {df_combined.shape}')
print(f'\nColumns: {list(df_combined.columns)}')

print('\nLabel distribution:')
label_dist = df_combined['label'].value_counts().sort_index()
for label, count in label_dist.items():
    print(f'  Label {label}: {count:>7,} ({count/len(df_combined)*100:>5.2f}%)')

print('\nClass balance:')
balance_ratio = label_dist.min() / label_dist.max()
print(f'  Ratio (minority/majority): {balance_ratio:.3f}')
if balance_ratio < 0.5:
    print('  ⚠ Dataset is imbalanced - consider using class weights or resampling')
else:
    print('  ✓ Dataset is reasonably balanced')

print('\nSource distribution:')
for source, count in df_combined['source'].value_counts().items():
    print(f'  {source:15s}: {count:>7,} ({count/len(df_combined)*100:>5.2f}%)')

print('\nText statistics by label:')
print(df_combined.groupby('label')[['text_length', 'word_count']].describe())

FINAL DATASET SUMMARY
Total emails: 202,938
Shape: (202938, 13)

Columns: ['sender', 'receiver', 'date', 'urls', 'subject', 'body', 'label', 'source', 'subject_clean', 'body_clean', 'text', 'text_length', 'word_count']

Label distribution:
  Label 0: 108,624 (53.53%)
  Label 1:  94,314 (46.47%)

Class balance:
  Ratio (minority/majority): 0.868
  ✓ Dataset is reasonably balanced

Source distribution:
  TREC-05        :  55,210 (27.21%)
  TREC-07        :  53,757 (26.49%)
  CEAS-08        :  39,154 (19.29%)
  Enron          :  29,767 (14.67%)
  TREC-06        :  16,382 ( 8.07%)
  Assassin       :   5,809 ( 2.86%)
  Ling           :   2,859 ( 1.41%)

Text statistics by label:
      text_length                                                        \
            count         mean          std  min    25%     50%     75%   
label                                                                     
0        108624.0  2016.715109  4995.416027  9.0  516.0  1025.0  1997.0   
1         94314.0

## 8. Train/Validation/Test Split

In [21]:
# Prepare final dataset with text and metadata columns
df_final = df_combined[['sender', 'receiver', 'date', 'text', 'urls', 'label', 'source']].copy()

# Stratified split: 70% train, 15% validation, 15% test
print('Creating train/validation/test split...')
print('='*70)

# First split: 70% train, 30% temp (for val + test)
df_train, df_temp = train_test_split(
    df_final,
    test_size=0.30,
    stratify=df_final['label'],
    random_state=42
)

# Second split: 50% of temp for validation, 50% for test (15% each of total)
df_val, df_test = train_test_split(
    df_temp,
    test_size=0.50,
    stratify=df_temp['label'],
    random_state=42
)

print(f'Train set: {len(df_train):>7,} ({len(df_train)/len(df_final)*100:>5.2f}%)')
print(f'Val set:   {len(df_val):>7,} ({len(df_val)/len(df_final)*100:>5.2f}%)')
print(f'Test set:  {len(df_test):>7,} ({len(df_test)/len(df_final)*100:>5.2f}%)')
print(f'Total:     {len(df_final):>7,}')

print('\nLabel distribution in splits:')
print('\nTrain:')
print(df_train['label'].value_counts().sort_index())
print('\nValidation:')
print(df_val['label'].value_counts().sort_index())
print('\nTest:')
print(df_test['label'].value_counts().sort_index())

print('\nColumns in final datasets:')
print(list(df_final.columns))

Creating train/validation/test split...
Train set: 142,056 (70.00%)
Val set:    30,441 (15.00%)
Test set:   30,441 (15.00%)
Total:     202,938

Label distribution in splits:

Train:
label
0    76036
1    66020
Name: count, dtype: int64

Validation:
label
0    16294
1    14147
Name: count, dtype: int64

Test:
label
0    16294
1    14147
Name: count, dtype: int64

Columns in final datasets:
['sender', 'receiver', 'date', 'text', 'urls', 'label', 'source']


## 9. Save Processed Data

In [22]:
print('Saving processed datasets...')
print('='*70)

# Save to CSV
train_path = PROCESSED_PATH / 'train.csv'
val_path = PROCESSED_PATH / 'val.csv'
test_path = PROCESSED_PATH / 'test.csv'
full_path = PROCESSED_PATH / 'full_processed.csv'

df_train.to_csv(train_path, index=False)
df_val.to_csv(val_path, index=False)
df_test.to_csv(test_path, index=False)
df_final.to_csv(full_path, index=False)

print(f'✓ Train set saved to: {train_path}')
print(f'  Size: {train_path.stat().st_size / (1024*1024):.2f} MB')
print(f'✓ Validation set saved to: {val_path}')
print(f'  Size: {val_path.stat().st_size / (1024*1024):.2f} MB')
print(f'✓ Test set saved to: {test_path}')
print(f'  Size: {test_path.stat().st_size / (1024*1024):.2f} MB')
print(f'✓ Full dataset saved to: {full_path}')
print(f'  Size: {full_path.stat().st_size / (1024*1024):.2f} MB')

print('\n✓ All datasets saved successfully!')

Saving processed datasets...
✓ Train set saved to: ..\..\..\data\processed\email-detection\train.csv
  Size: 238.36 MB
✓ Validation set saved to: ..\..\..\data\processed\email-detection\val.csv
  Size: 51.71 MB
✓ Test set saved to: ..\..\..\data\processed\email-detection\test.csv
  Size: 51.80 MB
✓ Full dataset saved to: ..\..\..\data\processed\email-detection\full_processed.csv
  Size: 341.87 MB

✓ All datasets saved successfully!


## 10. Data Quality Report

In [23]:
# Create a summary report
report = f"""
{'='*70}
DATA PREPROCESSING REPORT
{'='*70}

INPUT DATA:
  - Total raw emails: {sum(len(df) for df in datasets.values()):,}
  - Datasets: {len(datasets)}

CLEANING:
  - Garbage labels removed: {total_removed:,}
  - Clean emails retained: {total_clean:,}
  - Retention rate: {total_clean/(total_clean+total_removed)*100:.2f}%

FINAL DATASET:
  - Total emails: {len(df_final):,}
  - Legitimate (0): {(df_final['label']==0).sum():,}
  - Phishing (1): {(df_final['label']==1).sum():,}
  - Class balance ratio: {balance_ratio:.3f}

SPLITS:
  - Train: {len(df_train):,} ({len(df_train)/len(df_final)*100:.1f}%)
  - Validation: {len(df_val):,} ({len(df_val)/len(df_final)*100:.1f}%)
  - Test: {len(df_test):,} ({len(df_test)/len(df_final)*100:.1f}%)

TEXT STATISTICS:
  - Average text length: {df_final['text'].str.len().mean():.0f} characters
  - Average word count: {df_final['text'].str.split().str.len().mean():.0f} words
  - Max text length: {df_final['text'].str.len().max():,} characters

OUTPUT FILES:
  - {train_path}
  - {val_path}
  - {test_path}
  - {full_path}

READY FOR BERT-LSTM MODEL TRAINING!
{'='*70}
"""

print(report)

# Save report
report_path = PROCESSED_PATH / 'preprocessing_report.txt'
with open(report_path, 'w') as f:
    f.write(report)

print(f'\n✓ Report saved to: {report_path}')


DATA PREPROCESSING REPORT

INPUT DATA:
  - Total raw emails: 206,800
  - Datasets: 7

CLEANING:
  - Garbage labels removed: 3,862
  - Clean emails retained: 202,938
  - Retention rate: 98.13%

FINAL DATASET:
  - Total emails: 202,938
  - Legitimate (0): 108,624
  - Phishing (1): 94,314
  - Class balance ratio: 0.868

SPLITS:
  - Train: 142,056 (70.0%)
  - Validation: 30,441 (15.0%)
  - Test: 30,441 (15.0%)

TEXT STATISTICS:
  - Average text length: 1613 characters
  - Average word count: 259 words
  - Max text length: 641,703 characters

OUTPUT FILES:
  - ..\..\..\data\processed\email-detection\train.csv
  - ..\..\..\data\processed\email-detection\val.csv
  - ..\..\..\data\processed\email-detection\test.csv
  - ..\..\..\data\processed\email-detection\full_processed.csv

READY FOR BERT-LSTM MODEL TRAINING!


✓ Report saved to: ..\..\..\data\processed\email-detection\preprocessing_report.txt


## Summary

### What was done:
1. ✅ Loaded 7 email datasets
2. ✅ Filtered out garbage labels (kept only 0 and 1)
3. ✅ Standardized schema with metadata (sender, receiver, date, text, urls, label, source)
4. ✅ Combined all datasets into one
5. ✅ Handled missing values
6. ✅ Preprocessed text (cleaned, combined subject + body into 'text' field)
7. ✅ Created stratified train/val/test splits (70/15/15)
8. ✅ Saved processed datasets

### Schema Details:
- **Datasets with full metadata** (sender, receiver, date, urls): Assassin, CEAS-08, TREC-05, TREC-06, TREC-07
- **Datasets with minimal schema** (text only): Enron, Ling
- All datasets combined with null values for missing metadata fields

### Next Steps:
1. Build BERT-LSTM model architecture
2. Implement BERT tokenization
3. Create data loaders (PyTorch/TensorFlow)
4. Train the model
5. Evaluate performance

### Files Created:
- `data/processed/email-detection/train.csv` - Training set (sender, receiver, date, text, urls, label, source)
- `data/processed/email-detection/val.csv` - Validation set
- `data/processed/email-detection/test.csv` - Test set
- `data/processed/email-detection/full_processed.csv` - Complete dataset
- `data/processed/email-detection/preprocessing_report.txt` - Summary report